In [ ]:
import pandas as pd
import csv
from collections import defaultdict
from datetime import datetime
import matplotlib.patches as mpatches
import matplotlib
import time
import math
from operator import itemgetter
from scipy.spatial import distance
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [ ]:
recipe = pd.read_csv('./data/recipe.csv')
recipe.tail()

In [24]:
recipe = recipe[['recipe_id', 'recipe_name']]
recipe.tail()

,recipe_id,recipe_name
1303,recipe-1304,맥적구이
1304,recipe-1305,채소어묵
1305,recipe-1306,주꾸미보리죽
1306,recipe-1307,오렌지삼겹찜
1307,recipe-1308,소고기볼게살타르


In [45]:
# user_id, rating, recipe_id
ratings = pd.read_csv('./data/ratings.csv')
ratings = ratings[['recipe_id', 'user_id', 'rating']]
ratings.tail()

,recipe_id,user_id,rating
49995,recipe-1308,user-87,3
49996,recipe-1308,user-90,5
49997,recipe-1308,user-95,3
49998,recipe-1308,user-96,1
49999,recipe-1308,user-99,3


In [46]:
# user_id
users = pd.read_csv('./data/user.csv')
users = users[['user_id']]
users.tail()

,user_id
95,user-96
96,user-97
97,user-98
98,user-99
99,user-100


In [47]:
user_num = ratings.user_id.unique().shape[0]
recipe_num = ratings.recipe_id.unique().shape[0]

print("유저의 수는 {}명 이고, \n레시피의 수는 {}개 입니다.".format(user_num, recipe_num))

유저의 수는 100명 이고, 
레시피의 수는 1308개 입니다.


In [49]:
size = ratings.groupby(['user_id']).size().reset_index(name='size')
top10 = size.sort_values(by=['size'], axis=0, ascending=False)
top10 = top10.head(10)

In [50]:
top10

,user_id,size
0,user-1,500
63,user-66,500
73,user-75,500
72,user-74,500
71,user-73,500
70,user-72,500
69,user-71,500
68,user-70,500
67,user-7,500
66,user-69,500


In [51]:
top10 = pd.merge(top10, ratings, on = ['user_id'], how = 'inner')

In [52]:
result_top10 = pd.merge(top10, users, on = 'user_id', how = 'inner')
result_top10

,user_id,size,recipe_id,rating
0,user-1,500,recipe-1,1
1,user-1,500,recipe-5,2
2,user-1,500,recipe-6,4
3,user-1,500,recipe-8,2
4,user-1,500,recipe-15,4
...,...,...,...,...
4995,user-69,500,recipe-1300,3
4996,user-69,500,recipe-1301,1
4997,user-69,500,recipe-1303,3
4998,user-69,500,recipe-1305,4


In [53]:
del result_top10['size']

In [54]:
ratings_result = pd.merge(ratings, user, on = 'user_id', how = 'inner')
ratings_result.tail()

,recipe_id,user_id,rating,Unnamed: 0
49995,recipe-1291,user-68,4,67
49996,recipe-1292,user-68,4,67
49997,recipe-1295,user-68,1,67
49998,recipe-1299,user-68,4,67
49999,recipe-1300,user-68,5,67


In [55]:
# 사용자 샘플 선출 -> pivot table 생성
UM_matrix_ds = ratings_result.pivot(columns='recipe_id', values='rating')
UM_matrix_ds.head(5)

recipe_id,recipe-1,recipe-10,recipe-100,recipe-1000,recipe-1001,recipe-1002,recipe-1003,recipe-1004,recipe-1005,recipe-1006,...,recipe-990,recipe-991,recipe-992,recipe-993,recipe-994,recipe-995,recipe-996,recipe-997,recipe-998,recipe-999
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
def distance_cosine(a, b) :
       return 1-distance.cosine(a, b)

In [57]:
def distance_correlation(a, b) :
       return 1-distance.correlation(a, b)

In [58]:
def distance_euclidean(a, b) :
       return 1/(1+distance.euclidean(a, b))

In [91]:
def nearest_neighbor_user(user, simFunc) :
    u1  =  UM_matrix_ds.loc[user].dropna()
    ratedIndex = u1.index
    nn = { }
    
    for uid, row in UM_matrix_ds.iterrows():
        interSectionU1 = [ ]
        interSectionU2 = [ ]
        if uid == user: continue
        
        for i in ratedIndex:
            if not math.isnan(row[i]):
                interSectionU1.append(u1[i])
                interSectionU2.append(row[i])
        interSectionLen = len(interSectionU1)
        
        if interSectionLen < 3: continue
        sim = simFunc(interSectionU1, interSectionU2)
        
        if not math.isnan(sim):nn[uid] = sim
                                                
    return sorted(nn.items(), key=itemgetter(1))[::-1]
#     return sorted(nn.items(), key=itemgetter(1))[:-(topN+1):-1]

In [ ]:
for user in result_top10.index :
      print('User {0} neighbors : {1}'.format(user, nearest_neighbor_user(int(user), distance_cosine)))

User 0 neighbors : []


KeyboardInterrupt: 

Exception ignored in: 'pandas._libs.lib.c_is_list_like'
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/abc.py", line 139, in __instancecheck__
    return _abc_instancecheck(cls, instance)
KeyboardInterrupt


User 1 neighbors : []
User 2 neighbors : []
User 3 neighbors : []
User 4 neighbors : []
User 5 neighbors : []
User 6 neighbors : []
User 7 neighbors : []
User 8 neighbors : []
User 9 neighbors : []
User 10 neighbors : []
User 11 neighbors : []
User 12 neighbors : []
User 13 neighbors : []
User 14 neighbors : []
User 15 neighbors : []
User 16 neighbors : []
User 17 neighbors : []
User 18 neighbors : []
User 19 neighbors : []
User 20 neighbors : []
User 21 neighbors : []
User 22 neighbors : []
User 23 neighbors : []
User 24 neighbors : []
User 25 neighbors : []
User 26 neighbors : []
User 27 neighbors : []


In [ ]:
for user in result_top10['reviewNo'] :
      print('User {0} neighbors : {1}'.format(user, nearest_neighbor_user(int(user), 3, distance_correlation)))

In [70]:
def predict_rating(userid, nn=100, simFunc=distance_cosine):
    neighbor = nearest_neighbor_user(userid, nn, simFunc)
    neighbor_id = [id for id, sim in neighbor]
    
    neighbor_recipe = UM_matrix_ds.loc[neighbor_id].dropna(1, how='all', thresh=1)
    neighbor_dict = (dict(neighbor))
    ret = []
    
    for recipe_id, row in neighbor_recipe.iteritems():
        jsum, wsum = 0, 0
        for v in row.dropna().iteritems():
            sim = neighbor_dict.get(v[0], 0)
            jsum += sim
            wsum += (v[1]*sim)
        ret.append([recipe_id, wsum/jsum])
        
    return ret

In [74]:
result_top10

,user_id,recipe_id,rating
0,user-1,recipe-1,1
1,user-1,recipe-5,2
2,user-1,recipe-6,4
3,user-1,recipe-8,2
4,user-1,recipe-15,4
...,...,...,...
4995,user-69,recipe-1300,3
4996,user-69,recipe-1301,1
4997,user-69,recipe-1303,3
4998,user-69,recipe-1305,4


In [85]:
temp = result_top10.iloc[0].name
temp

0

In [88]:
result_list = []
for i in range(10):
    user_id = result_top10.iloc[i].name
    recipe_id = result_top10.iloc[i].recipe_id
    predict = predict_rating(user_id, 300, distance_cosine)
    print(predict)
    for recipe in predict:
        if recipe_id == recipe[0]:
            result_list.append([(user_id), (recipe_id), recipe[1]])
            
resultdf = pd.DataFrame(result_list, columns=['user_id','recipe_id','rating'])
resultdf

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


,user_id,recipe_id,rating
